In [1]:
import logging

import numpy as np
import sympy as sp
sp.init_printing(use_unicode=True)

logging.basicConfig(level=logging.INFO)

In [137]:
class PerspectiveTool:
    def __init__(self): pass
    
    @staticmethod
    def find_config(A, B, A1, B1, V):
        a = np.linalg.det([V, A, A1])
        b = np.linalg.det([V, A, B1])
        c = np.linalg.det([V, B, A1])
        d = np.linalg.det([V, B, B1])

        return a, b, c, d

    @staticmethod
    def cal_image(config, param):
        m = np.array([
            [config[1], config[3]],
            [-config[0], -config[2]]
        ])
        return m.dot(param)

    @staticmethod
    def get_center_matrix(A, B, A1, B1, config):
        # Create the augemented matrix representing the system 3(a)-(d) in page 154,
        # which is used to find the center once we know the config and the pairs of base point
        AA1 = cal_line_coordinates(A, A1)
        AB1 = cal_line_coordinates(A, B1)
        BA1 = cal_line_coordinates(B, A1)
        BB1 = cal_line_coordinates(B, B1)

        return np.array([
            np.concatenate((AA1, [config[0]])),
            np.concatenate((AB1, [config[1]])),
            np.concatenate((BA1, [config[2]])),
            np.concatenate((BB1, [config[3]]))
        ])
    
    @staticmethod
    def get_perspective_matrix(config):
        return np.array(
        [
            [config[1], config[3]],
            [-config[0], -config[2]]            
        ])
    
    @staticmethod
    def test_conditions(A, B, A1, B1, V, config=None):
        cond1 = True; cond2 = True
        if not config:
            a, b, c, d = PerspectiveTool.find_config(A, B, A1, B1, V)
        else:
            a,b,c,d = config

        AA1 = cal_line_coordinates(A, A1)
        AB1 = cal_line_coordinates(A, B1)
        BA1 = cal_line_coordinates(B, A1)
        BB1 = cal_line_coordinates(B, B1)

        out1 = a * d - b * c
        out2 = a * np.linalg.det([AB1, BA1, BB1]) \
            + b * np.linalg.det([AA1, BA1, BB1]) \
            + c * np.linalg.det([AA1, AB1, BB1]) \
            + d * np.linalg.det([AA1, AB1, BA1])

        if out1 == 0:
            logging.info("Not pass 5(a) condition")
            cond1 = False

        if out2 != 0:
            logging.info("Not pass 5(c) condition")
            cond2 = False

        logging.info("Perspective configuration: a={}, b={}, c={}, d={}".format(a, b, c, d))

        return False if not cond1 and not cond2 else True    

    @staticmethod
    def find_center(A, B, A1, B1, config):
        # Not exactly return the center, but return the rref of the center matrix, from
        # which center can be obtained.
        cm = PerspectiveTool.get_center_matrix(A,B,A1,B1,config)
        logging.info(cm)
        return sp.Matrix(cm).rref()
        
    

def cal_line_coordinates(A, B):
    return np.array([
        A[1]*B[2] - A[2]*B[1], 
        A[2]*B[0] - A[0]*B[2], 
        A[0]*B[1] - A[1]*B[0]]) 
    
    
def is_collinear(A, B, C):
    # Check if two point on the same line
    d = np.linalg.det(np.array([A,B,C]))
    a = 10**(-8) # threshold
    if d == 0 or abs(d) < a:
        if d != 0:
            logging.info("determinant of square matrix [{}, {}, {}] = {}"
                         .format(A, B, C, d))
        return True
    else: 
        return False
    
def guess_linear_coefficients(A, B, C):
    # Return the row reduced echelon form of the matrix, which
    # suggests a pair of coefficients to describe C by the linear
    # combination of A and B
    C = np.array([C])
    m = np.array([A, B]).transpose()
    m = np.concatenate((m, C.T), axis=1)

    M = sp.Matrix(m)
    return M.rref()    

def is_parallel(A, B):
    # Check two vectors are parallel (or equal in the extended Euclidean plane)
    # See Calculus - Steward, section 12.4 for the explanation of the calculation
    s = 10**(-10)
    a = A.dot(B)
    A_length = np.linalg.norm(A)
    B_length = np.linalg.norm(B)

    if abs(a / (A_length * B_length)) > (1 - s):
        return True
    else: 
        return False

In [138]:
### Exercise 4.4

def ex441(msg=None):
    V = np.array([0, 1, 0])
    A = np.array([1, -1, 0])
    B = np.array([-2, 2, 1])
    A1 = np.array([1, 1, 0])
    B1 = np.array([1, 3, 1])

    config = PerspectiveTool.find_config(A, B, A1, B1, V)
    if msg:
        print("Ex 4.4.1")
        print("The configuration: {}".format(config))
    return config
    
def ex442():
    print("Ex 4.4.2")
    V = np.array([0, 1, 0])
    
    A = np.array([1, -1, 2])
    B = np.array([1, -1, 0])
    A1 = np.array([-1, 1, 1])
    B1 = np.array([2, 0, -1])
    
    p = np.array([-2,2,5]) # on the line l
    param2 = [5, -9]
    param1 = [8, 5] # relate to the point A and B in ex441
    
    config1 = ex441()
    img_param1 = PerspectiveTool.cal_image(config1, param1)
    p1 = np.array([[1, 1, 0], [1, 3, 1]]).transpose().dot(img_param1)
    
    config2 = PerspectiveTool.find_config(A, B, A1, B1, V)
    img_param2 = PerspectiveTool.cal_image(config2, param2)
    p2 = np.array([A1, B1]).transpose().dot(img_param2)
    
    if is_parallel(p1, p2):
        print("Two configurations return the same points, {}, {}".format(p1, p2))
    else:
        print("Two configurations do NOT return the same points, {}, {}".format(p1, p2))
    
    
def ex443():
    A = np.array([2,1,1])
    B = np.array([0,0,1])
    A1 = np.array([0,1,-1])
    B1 = np.array([1,0,0])
    
    config1 = [2,3,-2,3]
    m1 = sp.Matrix(PerspectiveTool.get_center_matrix(A, B, A1, B1, config1))

    
    config2 = [4,3,1,2]
    m2 = sp.Matrix(PerspectiveTool.get_center_matrix(A, B, A1, B1, config2))
    
    print("Ex 4.4.3")
    print("Solution of a)")
    print(m1.rref())    
    print("Solution of b)")
    print(m2.rref())

def ex444():
    A = np.array([1,1,-2])
    B = np.array([1,2,-3])
    A1 = np.array([1,1,0])
    B1 = np.array([0,1,1])    

#     A = np.array([2,1,1])
#     B = np.array([0,0,1])
#     A1 = np.array([0,1,-1])
#     B1 = np.array([1,0,0])
    
    AA1 = cal_line_coordinates(A, A1)
    AB1 = cal_line_coordinates(A, B1)
    BA1 = cal_line_coordinates(B, A1)
    BB1 = cal_line_coordinates(B, B1)
    
    p1 = np.linalg.det([AB1, BA1, BB1])
    p2 = -np.linalg.det([AA1, BA1, BB1])
    p3 = np.linalg.det([AA1, AB1, BB1])
    p4 = -np.linalg.det([AA1, AB1, BA1])
    
    print(AA1, AB1, BA1, BB1)
    print(p1, p2, p3, p4)

    
def ex445():
    A = np.array([1,2,1])
    B = np.array([1,1,0])
    A1 = np.array([-2,1,3])
    B1 = np.array([-2,1,0])    
    
    AA1 = cal_line_coordinates(A, A1)
    AB1 = cal_line_coordinates(A, B1)
    BA1 = cal_line_coordinates(B, A1)
    BB1 = cal_line_coordinates(B, B1)
    
    p1 = np.linalg.det([AB1, BA1, BB1])
    p2 = -np.linalg.det([AA1, BA1, BB1])
    p3 = np.linalg.det([AA1, AB1, BB1])
    p4 = -np.linalg.det([AA1, AB1, BA1])
    
    print(AA1, AB1, BA1, BB1)
    print("{},{},{},{}".format(p1, p2, p3, p4))    
    
    
def ex448():
    A1 = np.array([0,1,2])
    B1 = np.array([0,2,2])
    A2 = np.array([-1,0,-1])
    B2 = np.array([1,0,2])
    A3 = np.array([-1,1,0])
    B3 = np.array([2,1,0])
    
    V1 = np.array([1,-1,1])
    V2 = np.array([-1,1,1])
    V3 = np.array([1,1,1])
    
    config1 = PerspectiveTool.find_config(A1,B1,A2,B2,V1)
    config2 = PerspectiveTool.find_config(A2,B2,A3,B3,V2)  
    config3 = PerspectiveTool.find_config(A3,B3,A1,B1,V3)
    
    pm1 = PerspectiveTool.get_perspective_matrix(config1)
    pm2 = PerspectiveTool.get_perspective_matrix(config2)
    pm3 = PerspectiveTool.get_perspective_matrix(config3)
    
    param = [2,3]
    P = A1*param[0] + B1*param[1]
    img_param = (pm3.dot(pm2.dot(pm1))).dot(param)
    img_P = A1*img_param[0] + B1*img_param[1]
    
    
    return pm1, pm2, pm3, pm3.dot(pm2.dot(pm1)), is_parallel(P, img_P), P, img_P
    

def ex449():
    A1 = np.array([0,1,2])
    B1 = np.array([0,2,2])
    A2 = np.array([-1,0,-1])
    B2 = np.array([1,0,2])
    A3 = np.array([-1,1,0])
    B3 = np.array([2,1,0])
    
    V1 = np.array([1,4,1])
    V2 = np.array([1,4,2])
    V3 = np.array([1,2,1])
    
    config1 = PerspectiveTool.find_config(A1,B1,A2,B2,V1)
    config2 = PerspectiveTool.find_config(A2,B2,A3,B3,V2)
    config3 = PerspectiveTool.find_config(A3,B3,A1,B1,V3)
    
    pm1 = PerspectiveTool.get_perspective_matrix(config1)
    pm2 = PerspectiveTool.get_perspective_matrix(config2)
    pm3 = PerspectiveTool.get_perspective_matrix(config3)
    
    param = [2,3]
    P = A1*param[0] + B1*param[1]
    img_param = (pm3.dot(pm2.dot(pm1))).dot(param)
    img_P = A1*img_param[0] + B1*img_param[1]
    
    
    return pm1, pm2, pm3, pm3.dot(pm2.dot(pm1)), is_parallel(P, img_P), P, img_P

def ex4410():
    A1 = np.array([1,3,3])
    B1 = np.array([0,-1,-1])
    A2 = np.array([-1,5,-1])
    B2 = np.array([2,-1,2])
    A3 = np.array([-1,1,2])
    B3 = np.array([2,-2,1])
    
    U = np.array([2,0,3])
    V = np.array([2,4,1])    
    
    config1 = PerspectiveTool.find_config(A1,B1,A2,B2,U)
    config2 = PerspectiveTool.find_config(A2,B2,A3,B3,V)
    pm1 = PerspectiveTool.get_perspective_matrix(config1)
    pm2 = PerspectiveTool.get_perspective_matrix(config2)
    pm = pm2.dot(pm1)
    
    return pm

In [139]:
# ex441(True)
# ex442()
ex443()
# ex444()
# ex445()
# ex448()
# ex449()
# ex4410()

Ex 4.4.3
Solution of a)
(Matrix([
[1, 0, 0, 2],
[0, 1, 0, 3],
[0, 0, 1, 0],
[0, 0, 0, 0]]), (0, 1, 2))
Solution of b)
(Matrix([
[1, 0, 0, -1],
[0, 1, 0,  2],
[0, 0, 1, -1],
[0, 0, 0,  0]]), (0, 1, 2))


In [160]:
## Exercises 4.5

def ex451():
    A1 = np.array([0,1,2])
    B1 = np.array([0,-1,1])
    A2 = np.array([1,0,1])
    B2 = np.array([-1,0,2])
    A3 = np.array([1,1,0])
    B3 = np.array([2,1,0])
    
    U = np.array([1,-1,0])
    V = np.array([1,1,1])
    
    config1 = PerspectiveTool.find_config(A1,A2,B1,B2,U)
    config2 = PerspectiveTool.find_config(A2,B2,A3,B3,V)
    pm1 = PerspectiveTool.get_perspective_matrix(config1)
    pm2 = PerspectiveTool.get_perspective_matrix(config2)
    pro_m = pm2.dot(pm1)
    config = [-pro_m[1][0], pro_m[0][0], -pro_m[1][1], pro_m[0][1]] 
    cm = PerspectiveTool.get_center_matrix(A1,B1,A3,B3,config)
    if np.linalg.det(cm) == 0:
        return True
    else:
        return False
    
def ex453():
    A1 = np.array([0,1,0])
    B1 = np.array([0,0,1])
    A2 = np.array([0,0,1])
    B2 = np.array([1,-1,0])
    A3 = np.array([1,0,0]) # original point
#     A3 = np.array([1,-1,-1]) # The intersection between l' and l2
    B3 = np.array([0,1,1])
    
    cm1 = PerspectiveTool.get_center_matrix(A1,B1,A3,B3,[1,1,1,1])
    cm2 = PerspectiveTool.get_center_matrix(A3,B3,A2,B2,[1,1,1,1])
    
    # After manually compute, I got the configurations of the two perspectives 
    # between l1 and l', and between l' and l2
#     config1 = [-2/3, 1/6, 5/3, -1/6]
#     config2 = [18, 2, -3, -2]
    config1 = [-3,-0.2,1,0.2]
    config2 = [-17.5,-15,3/2,1]
    c1 = PerspectiveTool.find_center(A1,B1,A3,B3,config1)
    c2 = PerspectiveTool.find_center(A3,B3,A2,B2,config2)
    pm1 = PerspectiveTool.get_perspective_matrix(config1)
    pm2 = PerspectiveTool.get_perspective_matrix(config2)
    pm = pm2.dot(pm1)
    
    # Given the center solution, find the confugurations, and compare with those I found.
#     U = np.array([1,0,1])
#     V = np.array([3,2,3])
#     config11 = PerspectiveTool.find_config(A1,B1,A3,B3,U)
#     config21 = PerspectiveTool.find_config(A3,B3,A2,B2,V)
#     pm11 = PerspectiveTool.get_perspective_matrix(config11)
#     pm21 = PerspectiveTool.get_perspective_matrix(config21)
#     pm_test = pm21.dot(pm11)
    
    return cm2,cm1,c1,c2,pm

def ex454():
    A1 = np.array([0,1,0])
    B1 = np.array([0,0,1])
    A2 = np.array([0,0,1])
    B2 = np.array([1,-1,0])
    A3 = np.array([1,-1,-1]) # The intersection between l' and l2
    B3 = np.array([0,1,1])
    
    cm1 = PerspectiveTool.get_center_matrix(A1,B1,A3,B3,[1,1,1,1])
    cm2 = PerspectiveTool.get_center_matrix(A3,B3,A2,B2,[1,1,1,1])
    
    # After manually compute, I got the configurations of the two perspectives 
    # between l1 and l', and between l' and l2
    config1 = [0,-3/5,-1,3/5]
    config2 = [-10/3,-5,3,5]
    c1 = PerspectiveTool.find_center(A1,B1,A3,B3,config1)
    c2 = PerspectiveTool.find_center(A3,B3,A2,B2,config2)
    pm1 = PerspectiveTool.get_perspective_matrix(config1)
    pm2 = PerspectiveTool.get_perspective_matrix(config2)
    pm = pm2.dot(pm1)
    return cm2,cm1,c1,c2,pm

In [161]:
# ex451()
ex453()
# ex454()

INFO:root:[[ 0.   0.  -1.  -3. ]
 [ 1.   0.   0.  -0.2]
 [ 0.   1.   0.   1. ]
 [-1.   0.   0.   0.2]]
INFO:root:[[  0.   -1.    0.  -17.5]
 [  0.    0.   -1.  -15. ]
 [  1.    0.    0.    1.5]
 [  1.    1.   -1.    1. ]]


(array([[ 0, -1,  0,  1],
        [ 0,  0, -1,  1],
        [ 1,  0,  0,  1],
        [ 1,  1, -1,  1]]),
 array([[ 0,  0, -1,  1],
        [ 1,  0,  0,  1],
        [ 0,  1,  0,  1],
        [-1,  0,  0,  1]]),
 (Matrix([
  [  1,   0, 0, -0.2],
  [0.0,   1, 0,  1.0],
  [0.0, 0.0, 1,  3.0],
  [  0,   0, 0,    0]]),
  (0, 1, 2)),
 (Matrix([
  [1, 0, 0, 0],
  [0, 1, 0, 0],
  [0, 0, 1, 0],
  [0, 0, 0, 1]]),
  (0, 1, 2, 3)),
 array([[ 6., -4.],
        [-8.,  5.]]))

In [95]:
a[1]

array([-7., -5.])

In [150]:
A1 = np.array([0,1,0])
B1 = np.array([0,0,1])
A2 = np.array([0,0,1])
B2 = np.array([1,-1,0])
A3 = np.array([1,0,0]) # original point
B3 = np.array([0,1,1])

PerspectiveTool.get_center_matrix(A3,B3,A2,B2,[1,1,1,1])
PerspectiveTool.get_center_matrix(A1,B1,A3,B3,[1,1,1,1])

array([[ 0,  0, -1,  1],
       [ 1,  0,  0,  1],
       [ 0,  1,  0,  1],
       [-1,  0,  0,  1]])

In [131]:
A = PerspectiveTool.get_perspective_matrix()
B = np.array([
    [1/6, -1/6],
    [2/3, -5/3]
])
A.dot(B)

array([[-1.,  3.],
       [-1., -2.]])

In [132]:
sp.Matrix(B).rref()

⎛⎡1  0⎤        ⎞
⎜⎢    ⎥, (0, 1)⎟
⎝⎣0  1⎦        ⎠

In [133]:
sp.Matrix(A).rref()

⎛⎡1  0⎤        ⎞
⎜⎢    ⎥, (0, 1)⎟
⎝⎣0  1⎦        ⎠

In [136]:
A.dot([1,2])

array([ -2, -12])

In [156]:
sp.Matrix([
    [1,5/2,-1,0,-1],
    [-1,-5/2,0,-1,3],
    [-1,0,3/2,0,-1],
    [1,0,0,3/2,-2]
]).rref()

⎛⎡1  0  0  1.5   -2.0⎤           ⎞
⎜⎢                   ⎥           ⎟
⎜⎢0  1  0  -0.2  -0.4⎥           ⎟
⎜⎢                   ⎥, (0, 1, 2)⎟
⎜⎢0  0  1  1.0   -2.0⎥           ⎟
⎜⎢                   ⎥           ⎟
⎝⎣0  0  0   0     0  ⎦           ⎠

In [157]:
 -3.5*5

In [159]:
-15*3/2